# 利用Keras探索皮马人糖尿病数据集

## 目录
- [1.数据集介绍](#1.-数据集介绍)
- [2.导入数据集](#2.-导入数据集)
- [3.查看数据信息](#3.-查看数据信息)
- [4.使用Keras建立神经网络](#4.-使用Keras建立神经网络)
    - [4.1 定义模型](#4.1-定义模型)
- [5.测试神经网络](#5.-测试神经网络)  
    - [5.1 口算神经网络](#5.1-口算神经网络)
    - [5.2 分割数据](#5.2-分割数据)
        - [5.2.1 自动验证](#5.2.1-自动验证)
        - [5.2.2 手工验证](#5.2.2-手工验证)
        - [4.2.3 K折交叉验证](#5.2.3-手工K折交叉验证)
- [6.使用Scikit-Learn调用Keras的模型](#6.-使用Scikit-Learn调用Keras的模型)
    - [6.1 使用交叉验证检验深度学习模型](#6.1-使用交叉验证检验深度学习模型)
    - [6.2 使用网格搜索调整深度学习模型的参数](#6.2-使用网格搜索调整深度学习模型的参数)    


## 1. 数据集介绍
该数据集涵盖了皮马人的医疗记录，以及过去5年内是否有糖尿病，所有的数据都以数字的形式呈现。需要解决的问题是，判断一个instance是否有糖尿病（是为1否为0）。这显然是一个**二分类问题**。该数据集中有8个属性及1个类别，表示如下：

- 怀孕次数 --- Number of times pregnant
- 2小时口服葡萄糖耐量试验中的血浆葡萄糖浓度 --- Plasma glucose concentration a 2 hours in an oral glucose tolerance test
    舒张压（毫米汞柱）--- Diastolic blood pressure (mm Hg)
- 2小时血清胰岛素（mu U/ml) --- 2-Hour serum insulin (mu U/ml)
- 三头肌皮褶厚度 (毫米) --- Triceps skin fold thickness (mm)
- 体重指数（BMI）--- Body mass index (weight in kg/(height in m)^2)
- 糖尿病血系功能 --- Diabetes pedigree function
- 年龄（年）--- Age (years)
- 类别：过去5年内是否有糖尿病 --- Class variable (0 or 1)

## 2. 导入数据集

In [4]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

In [50]:
data = pd.read_csv('../data/pima-indians-diabetes.csv', header=None)

## 3. 查看数据信息

In [51]:
data.head()

,0,1,2,3,4,5,6,7,8
0,0,1,2,3,4,5.0,6.000,7,8
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0


In [9]:
data.describe()

,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [10]:
data.shape

(768, 9)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
0    768 non-null int64
1    768 non-null int64
2    768 non-null int64
3    768 non-null int64
4    768 non-null int64
5    768 non-null float64
6    768 non-null float64
7    768 non-null int64
8    768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


## 4. 使用Keras建立神经网络

### 4.1 定义模型
**Keras的模型由层构成**：我们建立一个Sequential模型，一层层加入神经元。第一步是确定输入层的数目正确：在创建模型时用input_dim参数确定。例如，有8个输入变量，就设成8。

隐层怎么设置？这个问题很难回答，需要慢慢试验。一般来说，如果网络够大，即使存在问题也不会有影响。这个例子里我们用3层全连接网络。

全连接层用Dense类定义：第一个参数是本层神经元个数，然后是初始化方式和激活函数。这里的初始化方法是0到0.05的连续型均匀分布（uniform），Keras的默认方法也是这个。也可以用高斯分布进行初始化（normal）。

前两层的激活函数是线性整流函数relu，最后一层的激活函数是S型函数sigmoid。之前大家喜欢用S型和正切函数，但现在线性整流函数效果更好。为了保证输出是0到1的概率数字，最后一层的激活函数是S型函数，这样映射到0.5的阈值函数也容易。前两个隐层分别有12和8个神经元，最后一层是1个神经元（是否有糖尿病）。

In [16]:
from keras.models import Sequential
from keras.layers import Dense

使用随机梯度下降时最好固定随机数种子，这样你的代码每次运行的结果都一致。这种做法在演示结果、比较算法或debug时特别有效。你可以随便选种子：

In [17]:
seed = 7
np.random.seed(seed)

In [52]:
X=data.iloc[:,0:8]
Y=data.iloc[:,8]

开始创建模型

In [30]:
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='normal', activation='sigmoid'))

In [53]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, nb_epoch=150, batch_size=10)

Epoch 1/150
769/769 [==============================] - 0s 566us/step - loss: 0.2924 - acc: 0.8023
Epoch 2/150
769/769 [==============================] - 0s 122us/step - loss: 0.2781 - acc: 0.8218
Epoch 3/150
769/769 [==============================] - 0s 120us/step - loss: 0.2886 - acc: 0.7984
Epoch 4/150
769/769 [==============================] - 0s 120us/step - loss: 0.2703 - acc: 0.8101
Epoch 5/150
769/769 [==============================] - 0s 121us/step - loss: 0.2691 - acc: 0.8153
Epoch 6/150
769/769 [==============================] - 0s 126us/step - loss: 0.2794 - acc: 0.8075
Epoch 7/150
769/769 [==============================] - 0s 124us/step - loss: 0.2739 - acc: 0.8036
Epoch 8/150
769/769 [==============================] - 0s 120us/step - loss: 0.2842 - acc: 0.7958
Epoch 9/150
769/769 [==============================] - 0s 124us/step - loss: 0.2643 - acc: 0.8101
Epoch 10/150
769/769 [==============================] - 0s 124us/step - loss: 0.2796 - acc: 0.8036
Epoch 11/150
769/76

我们把测试数据拿出来检验一下模型的效果。注意这样不能测试在新数据的预测能力。应该将数据分成训练和测试集。

调用模型的evaluation()方法，传入训练时的数据。输出是平均值，包括平均误差和其他的数据，例如准确度。

In [54]:
scores = model.evaluate(X,Y)
print '%s: %.3f' % (model.metrics_names[1], scores[1]*100)

769/769 [==============================] - 0s 75us/step
acc: 82.575


## 5. 测试神经网络

深度学习有很多参数要调：大部分都是拍脑袋的。所以测试特别重要：本章我们讨论几种测试方法。本章将：

- 使用Keras进行自动验证
- 使用Keras进行手工验证
- 使用Keras进行K折交叉验证

### 5.1 口算神经网络
创建神经网络时有很多参数：很多时候可以从别人的网络上抄，但是最终还是需要一点点做实验。无论是网络的拓扑结构（层数、大小、每层类型）还是小参数（损失函数、激活函数、优化算法、训练次数）等。

一般深度学习的数据集都很大，数据有几十万乃至几亿个。所以测试方法至关重要。

### 5.2 分割数据
数据量大和网络复杂会造成训练时间很长，所以需要将数据分成训练、测试或验证数据集。Keras提供两种办法：

自动验证
手工验证
#### 5.2.1 自动验证
Keras可以将数据自动分出一部分，每次训练后进行验证。在训练时用**validation_split**参数可以指定验证数据的比例，一般是总数据的20%或者33%。

In [55]:
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, verbose=False)
scores = model.evaluate(X,Y)
print '%s: %.3f' % (model.metrics_names[1], scores[1]*100)

769/769 [==============================] - 0s 17us/step
acc: 81.274


#### 5.2.2 手工验证
Keras也可以手工进行验证。我们定义一个**train_test_split**函数，将数据分成2：1的测试和验证数据集。在调用fit()方法时需要加入**validation_data**参数作为验证数据，数组的项目分别是输入和输出数据。

In [45]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.33, random_state=seed)

In [56]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), nb_epoch=150, batch_size=10, verbose=False)
scores = model.evaluate(X,Y)
print '%s: %.3f' % (model.metrics_names[1], scores[1]*100)

769/769 [==============================] - 0s 17us/step
acc: 82.965


#### 5.2.3 手工K折交叉验证
机器学习的金科玉律是K折验证，以验证模型对未来数据的预测能力。K折验证的方法是：将数据分成K组，留下1组验证，其他数据用作训练，直到每种分发的性能一致。

深度学习一般不用交叉验证，因为对算力要求太高。例如，K折的次数一般是5或者10折：每组都需要训练并验证，训练时间成倍上升。然而，如果数据量小，交叉验证的效果更好，误差更小。

scikit-learn有StratifiedKFold类，我们用它把数据分成10组。抽样方法是分层抽样，尽可能保证每组数据量一致。然后我们在每组上训练模型，使用verbose=0参数关闭每轮的输出。训练后，Keras会输出模型的性能，并存储模型。最终，Keras输出性能的平均值和标准差，为性能估算提供更准确的估计。

**当然，这是种繁琐的做法。我们可以直接使用scikit-learn去调用keras完成K-fold validation.**

## 6. 使用Scikit-Learn调用Keras的模型
Keras为scikit-learn封装了KerasClassifier和KerasRegressor方便我们对模型进行调参。

### 6.1 使用交叉验证检验深度学习模型
Keras的KerasClassifier和KerasRegressor两个类接受build_fn参数，传入编译好的模型。我们加入nb_epoch=150和batch_size=10这两个参数：这两个参数会传入模型的fit()方法。我们用scikit-learn的StratifiedKFold类进行10折交叉验证，测试模型在未知数据的性能，并使用cross_val_score()函数检测模型，打印结果。

In [62]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score


def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1,init='normal', activation='sigmoid'))
    
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

X=data.iloc[:,0:8]
Y=data.iloc[:,8]

model = KerasClassifier(build_fn=create_model, nb_epoch=150, batch_size=10)
kfold = StratifiedKFold(y=Y, n_folds=3, shuffle=True, random_state=seed)

results = cross_val_score(model, X, Y, cv=kfold)
print 'results:', results
print 'mean result:', results.mean()


Epoch 1/1
257/257 [==============================] - 0s 1ms/step
Epoch 1/1
257/257 [==============================] - 0s 1ms/step
Epoch 1/1
255/255 [==============================] - 0s 2ms/step
results: [0.64980545 0.64980545 0.65098039]
mean result: 0.6501970970798299


## 6.2 使用网格搜索调整深度学习模型的参数
使用scikit-learn封装Keras的模型十分简单。进一步想：我们可以给fit()方法传入参数，KerasClassifier的build_fn方法也可以传入参数。可以利用这点进一步调整模型。

我们用网格搜索测试不同参数的性能：create_model()函数可以传入optimizer和init参数，虽然都有默认值。那么我们可以用不同的优化算法和初始权重调整网络。具体说，我们希望搜索：

- 优化算法：搜索权重的方法
- 初始权重：初始化不同的网络
- 训练次数：对模型训练的次数
- 批次大小：每次训练的数据量
- 所有的参数组成一个字典，传入scikit-learn的GridSearchCV类：GridSearchCV会对每组参数（2×3×3×3）进行训练，进行3折交叉检验。

计算量巨大：耗时巨长。如果模型小还可以取一部分数据试试。比如我们这里使用的模型，网络和数据集都不大（1000个数据内，9个参数）。最后scikit-learn会输出最好的参数和模型，以及平均值。

In [66]:
from sklearn.grid_search import GridSearchCV

def create_model(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim=8, init=init, activation='relu'))
    model.add(Dense(8, init=init, activation='relu'))
    model.add(Dense(1, init=init, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

X=data.iloc[:,0:8]
Y=data.iloc[:,8]   

model = KerasClassifier(build_fn=create_model)

optimizers = ['rmsprop', 'adam']
inits = ['glorot_uniform', 'uniform', 'normal']
epoches = [50, 100, 150]
batches = [5, 10, 20]

param_grid= dict(optimizer=optimizers, nb_epoch=epoches, batch_size=batches, init=inits)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
results = grid.fit(X, Y)

print results



Epoch 1/1
257/257 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
257/257 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
257/257 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
257/257 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
257/257 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
257/257 [======

Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
257/257 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
257/257 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
257/257 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
257/257 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
257/257 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [======

In [70]:
print("Best: %f using %s" % (results.best_score_, results.best_params_))

Best: 0.659298 using {'init': 'normal', 'optimizer': 'adam', 'nb_epoch': 100, 'batch_size': 20}


通过上面的计算，我们得到了最佳模型的参数:
- Best: 0.659298 using {'init': 'normal', 'optimizer': 'adam', 'nb_epoch': 100, 'batch_size': 20}

In [69]:
for params, mean_score, scores in results.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

0.637165 (0.039419) with: {'init': 'glorot_uniform', 'optimizer': 'rmsprop', 'nb_epoch': 50, 'batch_size': 5}
0.488849 (0.105365) with: {'init': 'glorot_uniform', 'optimizer': 'adam', 'nb_epoch': 50, 'batch_size': 5}
0.586439 (0.047494) with: {'init': 'glorot_uniform', 'optimizer': 'rmsprop', 'nb_epoch': 100, 'batch_size': 5}
0.529249 (0.024535) with: {'init': 'glorot_uniform', 'optimizer': 'adam', 'nb_epoch': 100, 'batch_size': 5}
0.638527 (0.020554) with: {'init': 'glorot_uniform', 'optimizer': 'rmsprop', 'nb_epoch': 150, 'batch_size': 5}
0.520073 (0.045074) with: {'init': 'glorot_uniform', 'optimizer': 'adam', 'nb_epoch': 150, 'batch_size': 5}
0.650241 (0.025869) with: {'init': 'uniform', 'optimizer': 'rmsprop', 'nb_epoch': 50, 'batch_size': 5}
0.650241 (0.025869) with: {'init': 'uniform', 'optimizer': 'adam', 'nb_epoch': 50, 'batch_size': 5}
0.654147 (0.029988) with: {'init': 'uniform', 'optimizer': 'rmsprop', 'nb_epoch': 100, 'batch_size': 5}
0.650241 (0.025869) with: {'init': 'un

In [75]:
# st: 0.659298 using {'init': 'normal', 'optimizer': 'adam', 'nb_epoch': 100, 'batch_size': 20}
model = Sequential()
model.add(Dense(12, input_dim=8, init='normal', activation='relu'))
model.add(Dense(8, init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.33, random_state=seed)
model.fit(X_train, y_train, validation_data=(X_test,y_test), nb_epoch=100, batch_size=20, verbose=False)
scores = model.evaluate(X,Y)
print '%s: %.3f' % (model.metrics_names[1], scores[1]*100)

769/769 [==============================] - 0s 67us/step
acc: 72.432
